In [6]:
import requests
a = 0.4
b = 0.2
float(requests.get(f"http://ramcdougal.com/cgi-bin/error_function.py?a={a}&b={b}", headers={"User-Agent": "MyScript"}).text)

1.294915

In [ ]:
import random
import math

BASE_URL = "http://ramcdougal.com/cgi-bin/error_function.py"
HEADERS = {"User-Agent": "MyScript"}  

def get_error(a, b):
    resp = requests.get(
        BASE_URL,
        params={"a": a, "b": b},
        headers=HEADERS
    )
    resp.raise_for_status()
    return float(resp.text.strip())

def estimate_gradient(a, b, h=1e-3):
    # partial wrt a
    if a - h >= 0 and a + h <= 1:
        Ea_plus  = get_error(a + h, b)
        Ea_minus = get_error(a - h, b)
        dEa = (Ea_plus - Ea_minus) / (2 * h)
    elif a - h < 0:  # near lower bound
        Ea_plus  = get_error(a + h, b)
        Ea       = get_error(a, b)
        dEa = (Ea_plus - Ea) / h
    else:            # near upper bound
        Ea       = get_error(a, b)
        Ea_minus = get_error(a - h, b)
        dEa = (Ea - Ea_minus) / h

    # partial wrt b
    if b - h >= 0 and b + h <= 1:
        Eb_plus  = get_error(a, b + h)
        Eb_minus = get_error(a, b - h)
        dEb = (Eb_plus - Eb_minus) / (2 * h)
    elif b - h < 0:
        Eb_plus = get_error(a, b + h)
        Eb      = get_error(a, b)
        dEb = (Eb_plus - Eb) / h
    else:
        Eb      = get_error(a, b)
        Eb_minus = get_error(a, b - h)
        dEb = (Eb - Eb_minus) / h

    return dEa, dEb

def gradient_descent_2d(
    a0,
    b0,
    alpha=0.1,
    h=1e-3,
    tol_grad=1e-4,
    tol_param=1e-5,
    max_iters=200
):
    a, b = a0, b0
    E = get_error(a, b)

    for it in range(max_iters):
        dEa, dEb = estimate_gradient(a, b, h=h)

        # gradient norm as a stopping measure
        grad_norm = math.sqrt(dEa**2 + dEb**2)
        if grad_norm < tol_grad:
            break

        # gradient descent step
        new_a = a - alpha * dEa
        new_b = b - alpha * dEb

        # project back into [0, 1]
        new_a = min(1.0, max(0.0, new_a))
        new_b = min(1.0, max(0.0, new_b))

        # check parameter change
        delta = math.sqrt((new_a - a)**2 + (new_b - b)**2)
        a, b = new_a, new_b
        new_E = get_error(a, b)

        if delta < tol_param:
            E = new_E
            break

        E = new_E

    return a, b, E, it + 1


In [15]:
a_opt, b_opt, err_opt, n = gradient_descent_2d(0.7, 0.3)
print(a_opt, b_opt, err_opt, n)

0.7119987415000189 0.1690137364999897 1.00000000057 11


In [17]:
starts = [
    (0.1, 0.1),
    (0.9, 0.9),
    (0.1, 0.9),
    (0.9, 0.1),
    (0.5, 0.5),
    (random.random(), random.random())
]

results = []

for (a0, b0) in starts:
    a_opt, b_opt, err_opt, n = gradient_descent_2d(a0, b0)
    results.append((a0, b0, a_opt, b_opt, err_opt, n))

for r in results:
    print(f"Start=({r[0]:.3f},{r[1]:.3f}) -> "
          f"a={r[2]:.6f}, b={r[3]:.6f}, err={r[4]:.6f}, iters={r[5]}")

Start=(0.100,0.100) -> a=0.215992, b=0.688960, err=1.100000, iters=44
Start=(0.900,0.900) -> a=0.216047, b=0.689014, err=1.100000, iters=44
Start=(0.100,0.900) -> a=0.215981, b=0.689035, err=1.100000, iters=40
Start=(0.900,0.100) -> a=0.712008, b=0.168997, err=1.000000, iters=12
Start=(0.500,0.500) -> a=0.216038, b=0.688975, err=1.100000, iters=41
Start=(0.852,0.160) -> a=0.712015, b=0.168999, err=1.000000, iters=11
